# libs

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from scipy import spatial
import glob

# all zipcodes

In [2]:
zipcode_df = pd.read_csv('data/processed/zipcode.csv',index_col=[0])

zipcode_df = zipcode_df.rename(columns={
    'Latitude':'latitude',
    'Longitude':'longitude',
})

# all df_pararius

In [3]:
all_files = glob.glob("data/temp/df_pararius_*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=[0])
    li.append(df)

df_pararius = pd.concat(li, axis=0, ignore_index=True)

# What is new?

In [4]:
temp1 = df_pararius['postcode'].unique()
temp2 = zipcode_df['postcode'].unique()
cep = list(set(temp1) - set(temp2))
print(len(cep))

20


In [5]:
geolocator = Nominatim(user_agent="myGeocoder")
post_code=[]
erro_code=[]

for zipcode in cep[:]:
    try:
        dict_zip = {
            'zipcode':zipcode, 
            'geolocator':geolocator.geocode(zipcode).address, 
            'latitude':geolocator.geocode(zipcode).latitude, 
            'longitude':geolocator.geocode(zipcode).longitude, 
        }
        post_code.append(dict_zip)
        print('bom - ',zipcode)

    except:
        print('erro - ',zipcode)
        erro_code.append(zipcode)

bom -  1066LH
bom -  2517LB
bom -  3545DC
bom -  5521WX
bom -  1019VR
bom -  2611DV
bom -  1341CL
bom -  2592EX
bom -  7773DG
erro -  2595BX
bom -  2265BT
bom -  3648LC
bom -  3253MS
bom -  3541HA
bom -  4711SJ
bom -  2553DN
bom -  3931XZ
bom -  5961RV
erro -  2595BW
bom -  3015GL


In [6]:
zipcode = pd.DataFrame(post_code)
splited = zipcode['geolocator'].str.split(",",expand=True)
zipcode['Plaats'] = splited[0]
zipcode['Provincie'] = splited[1]
zipcode = zipcode.drop(['geolocator'],1)
zipcode

<ipython-input-6-484058219571>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  zipcode = zipcode.drop(['geolocator'],1)


,zipcode,latitude,longitude,Plaats,Provincie
0,1066LH,52.341948,4.826624,Amsterdam,Noord-Holland
1,2517LB,52.085374,4.275228,Den Haag,Zuid-Holland
2,3545DC,52.077211,5.065480,Utrecht,Nederland
3,5521WX,51.356630,5.330286,Eersel,Noord-Brabant
4,1019VR,52.368184,4.954946,Amsterdam,Noord-Holland
5,2611DV,52.005150,4.356429,Delft,Zuid-Holland
6,1341CL,52.358668,5.278395,Almere,Flevoland
7,2592EX,52.085850,4.356995,Den Haag,Zuid-Holland
8,7773DG,52.582108,6.603512,Hardenberg,Overijssel
9,2265BT,52.081247,4.393413,Leidschendam,Leidschendam-Voorburg


# Manual Coordinates

## Not Found

In [7]:
erro_code

['2595BX', '2595BW']

## Add manually [Source](https://postcodebijadres.nl/2511EX)

In [8]:
# df_2511EX = pd.DataFrame({})
# df_2511EX['Plaats'] = ['Schiphol']
# df_2511EX['Provincie'] = ['Noord-Holland']
# df_2511EX['latitude'] = 52.2921522
# df_2511EX['longitude'] = 4.7552736
# df_2511EX['postcode'] = '1118LL'
# zipcode_df = pd.concat([zipcode_df,df_2511EX],0)
# zipcode_df.tail()

# Save list zipcodes

In [9]:
zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)

<ipython-input-9-815d2116adb3>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode_df = pd.concat([zipcode_df,zipcode],0).drop_duplicates(subset=['postcode']).reset_index(drop=True)


In [10]:
zipcode_df = zipcode_df.drop(['zipcode'], axis=1)

# Add Coord into df_pararius

In [11]:
zipcode_df = zipcode_df.rename(columns={'zipcode':'postcode'})
df_pararius_coo=pd.merge(df_pararius, zipcode_df, on='postcode',how='left')

# Final info

In [12]:
df_pararius_coo = df_pararius_coo[df_pararius_coo['date'] == df_pararius_coo['date'].max()].reset_index(drop=True)

In [13]:
df_pararius_coo = df_pararius_coo.fillna(0).drop_duplicates(subset=['irl']).reset_index(drop=True)

In [14]:
df_pararius_coo.loc[df_pararius_coo['interior']==0, 'interior'] = 'Upholstered'
df_pararius_coo.loc[df_pararius_coo['status']==0, 'status'] = 'Free'

In [15]:
df_pararius_coo['price'] = df_pararius_coo['price'].astype(float)

In [16]:
# prepare link on streamlit
df_pararius_coo['url'] = 'https://www.pararius.com' + df_pararius_coo['irl']
df_pararius_coo['link'] = "<a target='_blank' href=" + (df_pararius_coo['url']).astype(str) + ">" + (df_pararius_coo['street']).astype(str) + "</a>"
df_pararius_coo['img'] = "<a href=" + df_pararius_coo['url'] + " target='blank'><img src=" + df_pararius_coo['image'] + "title='rent' width='150' height='100'/></a>"

In [17]:
try:
    stations = pd.read_csv('https://raw.githubusercontent.com/trainline-eu/stations/master/stations.csv', sep=';',low_memory=False)
    stations.to_csv('data/processed/stations.csv')
except:
    print('local')
    stations = pd.read_csv('data/processed/stations.csv')
    
stations = stations[stations['country']=='NL'][['name','latitude', 'longitude']]
stations = stations[stations['latitude'].notna()].reset_index(drop=True)


coordinates_1 = list(zip(df_pararius_coo['latitude'], df_pararius_coo['longitude']))
coordinates_2 = list(zip(stations['latitude'], stations['longitude']))
tree = spatial.KDTree(coordinates_2)


distance=[]
for i in range(len(df_pararius_coo)):
    teste = coordinates_1[i]
    distance.append(tree.query(teste)[0])

    
df_pararius_coo['train'] = distance

In [18]:
# clc_area   = ((df_pararius_coo['Living area'] - df_pararius_coo['Living area'].mean())/df_pararius_coo['Living area'].std())
# clc_room   = ((df_pararius_coo['Rooms'] - df_pararius_coo['Rooms'].mean())/df_pararius_coo['Rooms'].std())
# clc_garden = ((df_pararius_coo['garden area'] - df_pararius_coo['garden area'].mean())/df_pararius_coo['garden area'].std())
# clc_price  = ((df_pararius_coo['price'] - df_pararius_coo['price'].mean())/df_pararius_coo['price'].std())
# clc_train  = ((df_pararius_coo['train'] - df_pararius_coo['train'].mean())/df_pararius_coo['train'].std())

In [19]:
clc_area   = df_pararius_coo['Living area']
clc_room   = df_pararius_coo['Rooms']
clc_garden = df_pararius_coo['garden area']
clc_price  = df_pararius_coo['price']
clc_train  = df_pararius_coo['train'] 

In [20]:
# df_pararius_coo['deal'] = 100 * ((3*clc_area + clc_room + 2*clc_garden) / (3*clc_price + 2*clc_train))
df_pararius_coo['deal'] = ((clc_area + clc_room + clc_garden) / (clc_price + clc_train))

# Save results

In [21]:
df_pararius_coo['city'] = df_pararius_coo['irl'].str.split("/",expand=True)[2].str.replace("-", " ").str.title()

In [22]:
df_pararius_coo.to_csv('app/df_coo_pararius.csv')
zipcode_df.to_csv('data/processed/zipcode.csv')